In [1]:
import os
import sys
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

# Install Spark
!apt-get update
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"
os.environ["PYSPARK_PYTHON"] = sys.executable

sys.path.append(os.path.join(os.environ["SPARK_HOME"], "python"))
sys.path.append(os.path.join(os.environ["SPARK_HOME"], "python", "lib", "py4j-0.10.9.7-src.zip"))  # Adjust the version if needed

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
from pyspark.sql import SparkSession
import pandas as pd

# Initialize Spark
spark = SparkSession.builder.appName("GPR_Model").getOrCreate()

# Load data into Spark DataFrame
spark_df = spark.read.csv("cleaned_data.csv", header=True, inferSchema=True)

# Convert Spark DataFrame to Pandas for TensorFlow training
df = spark_df.toPandas()
df.head()

,Year,Month,GPR,GPRT,GPRA,GPRC_CAN,GPRC_USA,WTI
0,1986,1,135.36,137.67,166.02,0.56,3.38,22.93
1,1986,2,98.75,84.02,114.82,0.29,2.30,15.46
2,1986,3,98.68,85.13,117.56,0.24,2.43,12.61
3,1986,4,148.31,142.49,182.87,0.63,4.08,12.84
4,1986,5,117.39,126.68,114.20,0.57,2.92,15.38


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# df = pd.get_dummies(df)

# Define Features and Target
X = df.drop(columns=['WTI'])
y = df["WTI"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [4]:
# Define the number of input features
number_input_features = X_train_scaled.shape[1]

# Define the model
nn = tf.keras.models.Sequential()

# Add layers to the model
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=X_train_scaled.shape[1]))
nn.add(tf.keras.layers.Dense(units=50, activation="relu"))
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Check the structure of the model
nn.summary()

# Complile the model
nn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 80)                  │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 50)                  │           4,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,741 (18.52 KB)

 Trainable params: 4,741 (18.52 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
from tensorflow.keras.callbacks import EarlyStopping

# # Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# # Train the model
history = nn.fit(X_train_scaled, y_train,
                 validation_data=(X_test_scaled, y_test),
                 epochs=100, batch_size=16,
                 verbose=1)

Epoch 1/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 3141.0969 - mean_absolute_error: 47.9162 - val_loss: 2887.7397 - val_mean_absolute_error: 45.2864
Epoch 2/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 3219.5432 - mean_absolute_error: 47.9031 - val_loss: 2666.3049 - val_mean_absolute_error: 42.8235
Epoch 3/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 2808.6482 - mean_absolute_error: 43.8024 - val_loss: 2271.0979 - val_mean_absolute_error: 38.1751
Epoch 4/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2324.8127 - mean_absolute_error: 38.1050 - val_loss: 1703.6127 - val_mean_absolute_error: 31.4329
Epoch 5/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 1971.7231 - mean_absolute_error: 33.5996 - val_loss: 1107.6632 - val_mean_absolute_error: 24.5087
Epoch 6/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 1160.7288 - mean_absolute_error: 25.6443 - val_loss: 761.0685 - val_mean_absolute_error: 20.9430
Epoch 7/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - 

In [6]:
from sklearn.metrics import r2_score

# Evaluate on test data
y_pred = nn.predict(X_test_scaled)

# Compute R-squared
r2 = r2_score(y_test, y_pred)

# Print evaluation metrics
loss, mae = nn.evaluate(X_test_scaled, y_test)
print(f"""
Model Performance
-----------------
R² Score: {r2:.2f}
Test Loss: {loss:.4f}
Test MAE: {mae:.4f}
      """)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 144.1497 - mean_absolute_error: 7.6649

Model Performance
-----------------
R² Score: 0.82 
Test Loss: 151.5137 
Test MAE: 7.9284
      


In [7]:
import csv

# Save model results in CSV
model_results = [
    ["Experiment", "Hidden Layer 1", "Hidden Layer 2", "Batch Size", "Epochs", "Validation Loss", "Test MAE", "R² Score"],
    [1, 80, 50, 16, len(history.history['loss']), min(history.history['val_loss']), mae, r2]
]

# Write to CSV
with open("model_optimization_log.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(model_results)

print("\n🔍 Model results saved in 'model_optimization_log.csv'")


🔍 Model results saved in 'model_optimization_log.csv'
